In [1]:
!pip install isodate

In [4]:
import pandas as pd
import re
import isodate

# ==============================
# 1. Load dataset
# ==============================
df = pd.read_csv("/content/master_task1_datset_14.csv")

# ==============================
# 2. Define cleaning function
# ==============================
def clean_text(text):
    if pd.isna(text):
        return ""
    # Remove emojis (basic pattern)
    text = re.sub(r"[^\w\s,]", " ", text, flags=re.UNICODE)
    # Remove HTML tags
    text = re.sub(r"<.*?>", " ", text)
    # Remove special characters (#, @, |, [], etc.)
    text = re.sub(r"[#@|\[\]{}]", " ", text)
    # Convert to lowercase
    text = text.lower()
    # Strip extra whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text

# ==============================
# 3. Apply cleaning to multiple columns
# ==============================
cols_to_clean = [
    "title",
    "description",
    "defaultLanguage",
    "channel_title",
    "channel_description",
    "channel_country"
]

for col in cols_to_clean:
    if col in df.columns:
        df[col] = df[col].astype(str).apply(clean_text)

# ==============================
# 4. Remove rows with missing or invalid descriptions
# ==============================
before_rows = len(df)

# Remove rows where 'description' is NaN, empty, or literally 'nan'
df = df[
    df["description"].notna() &
    (df["description"].str.strip() != "") &
    (df["description"].str.lower() != "nan")
]

after_rows = len(df)
print(f"🧹 Removed {before_rows - after_rows} rows with missing or invalid descriptions")

# ==============================
# 5. Ensure title uniqueness
# ==============================
before = len(df)
df = df.drop_duplicates(subset=["id", "title"], keep="first")
after = len(df)
print(f"✅ Removed {before - after} duplicate titles")

# ==============================
# 6. Convert duration (ISO 8601 -> seconds)
# ==============================
def parse_duration_to_seconds(duration_str):
    """Parses an ISO 8601 duration string and returns total seconds."""
    try:
        if pd.isna(duration_str) or duration_str.strip() == "":
            return None
        duration = isodate.parse_duration(duration_str)
        return int(duration.total_seconds())
    except Exception as e:
        print(f"⚠️ Could not parse duration '{duration_str}': {e}")
        return None

if "duration" in df.columns:
    df["duration"] = df["duration"].apply(parse_duration_to_seconds)

# ==============================
# 7. Save cleaned dataset
# ==============================
df.to_csv("Master_task1_Cleaned_main.csv", index=False, encoding="utf-8")
print("💾 Cleaned dataset saved as Master_task1_Cleaned_main.csv")


🧹 Removed 55 rows with missing or invalid descriptions
✅ Removed 0 duplicate titles
💾 Cleaned dataset saved as Master_task1_Cleaned_main.csv


In [5]:
import pandas as pd
import re

# Load dataset
df = pd.read_csv("/content/master_dataset_task2_14.csv")

# ==============================
# 1. Define cleaning function for transcripts
# ==============================
def clean_transcript(text):
    if pd.isna(text):
        return ""

    # Remove [Music], [Applause], [Laughter], etc.
    text = re.sub(r"\[.*?\]", " ", text)

    # Remove timestamps (hh:mm:ss or mm:ss)
    text = re.sub(r"\b\d{1,2}:\d{2}(?::\d{2})?\b", " ", text)

    # Remove special characters & non-UTF symbols (keep words, numbers, spaces, punctuation)
    text = re.sub(r"[^a-zA-Z0-9\s,.?!']", " ", text)

    # Convert to lowercase
    text = text.lower()

    # Replace newlines with spaces
    text = text.replace("\n", " ")

    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text

# ==============================
# 2. Apply cleaning
# ==============================
if "transcript" in df.columns:
    df["transcript"] = df["transcript"].astype(str).apply(clean_transcript)

# ==============================
# 3. Drop missing/empty transcripts
# ==============================
df = df[df["transcript"].notna()]
df = df[df["transcript"].str.strip() != ""]

# Remove transcripts with less than 10 words
df = df[df["transcript"].apply(lambda x: len(x.split()) >= 10)]

# ==============================
# 4. Ensure id is valid (only one word, looks like YouTube ID)
# ==============================
df = df[df["id"].apply(lambda x: isinstance(x, str) and len(x.split()) == 1)]

# ==============================
# 5. Save cleaned dataset
# ==============================
df.to_csv("Master_task2_Cleaned_main.csv", index=False, encoding="utf-8")

print("💾 Cleaned dataset saved as Master_task2_Cleaned_main.csv")
print(f"✅ Final shape: {df.shape}")
display(df.head())

💾 Cleaned dataset saved as Master_task2_Cleaned_main.csv
✅ Final shape: (702, 3)


,id,transcript,Unnamed: 2
0,M4IHWsk-EAM,hey everyone welcome back to the channel i hop...,NaN
1,4Xh9DLUQCWs,so finally 7th august is here and we launching...,NaN
4,z9bJUPxzFOw,so let's continue with our playlist today for ...,NaN
5,cEadsbTeze4,so let's continue with the stack and q playlis...,NaN
6,NwBvene4Imo,so let's continue with our stack and q playlis...,NaN


In [7]:
import pandas as pd

# Load both datasets
dataset1 = pd.read_csv("/content/Master_task1_Cleaned_main.csv")   # metadata dataset
dataset2 = pd.read_csv("/content/Master_task2_Cleaned_main.csv")    # transcripts dataset

# Ensure consistent column names
# dataset1.rename(columns={"id": "video_id"}, inplace=True) # This line is not needed as 'id' is already the correct column name in dataset1
dataset1.rename(columns={"video_id": "id"}, inplace=True) # Rename 'video_id' to 'id' in dataset1 if it exists, assuming 'id' is the key column
dataset2.rename(columns={"video_id": "id"}, inplace=True) # Rename 'video_id' to 'id' in dataset2 if it exists

# Create a set of video_ids that have transcripts
transcript_ids = set(dataset2["id"].unique())

# Add new column in dataset1
dataset1["has_transcript"] = dataset1["id"].apply(lambda x: x in transcript_ids)

# Save the updated dataset
dataset1.to_csv("Master_Task1_withTranscriptFlag.csv", index=False, encoding="utf-8")

print("💾 Updated dataset saved as Master_Task1_withTranscriptFlag.csv")
print(dataset1[["id", "title", "has_transcript"]].head(10))

💾 Updated dataset saved as Master_Task1_withTranscriptFlag.csv
            id                                              title  \
0  r4loBN9ffls  data analyst jobs are cooked ð this is how you...   
1  yg_K45YtzSs                    data engineer vs data scientist   
2  xoptBxQvIT0  is big tech a trap why she quit her 200k softw...   
3  w9k1oXwiOJY                   5 data analyst projects you need   
4  CIfjV-q-g6M    ai engineer or software engineer ft anjali gama   
5  Uxr3sbB6h-c            3 reasons to not become a data engineer   
6  PcYZyb93Peg                data scientist vs software engineer   
7  n08nWEvdz-I        2m tech vp salary at amazon ft ethanevansvp   
8  4uHyKtxS1j8                     ai engineer vs prompt engineer   
9  z8kAQPEVanw  software engineer vs data scientist ft sajjaad...   

   has_transcript  
0            True  
1            True  
2            True  
3            True  
4            True  
5            True  
6            True  
7            True